In [1]:
import os
print(os.getcwd())

/workspaces/Ic-Rna-2024/fuction_ensemble_3/redes-ensemble-s/diversity


In [ ]:
import pandas as pd
import numpy as np

def diversity(file_name, file_paths):
    """
    Calcula o erro para um conjunto de redes fornecido e salva os resultados em um arquivo Excel.
    """
    # Carregar os dados de todos os arquivos e armazenar os valores de 'Z_pred'
    z_preds = [pd.read_excel(path)['Z_pred'].values.reshape(-1, 1) for path in file_paths]
    
    # Converter para array NumPy
    valores = np.array(z_preds)
    Z_media = valores.mean()

    # Criando grade de valores esperados
    x, y = np.meshgrid(np.linspace(-10, 10, 32), np.linspace(-10, 10, 32))
    Z = ((x + 2 * y - 7)**2 + (2 * x + y - 5)**2).reshape(-1, 1)

    # Cálculo do erro
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    # Criando DataFrame e salvando no Excel
    df_error = pd.DataFrame({'Error': error.flatten()})
    df_error.to_excel(f"{file_name}.xlsx", index=False)
    
    print(f'Arquivo {file_name}.xlsx salvo com sucesso!')

# Lista de arquivos de entrada
file_paths = [
    "1000_model_2_8_14.xlsx", "1000_model_4_3_13.xlsx",  "1000_model_8_4_3.xlsx", "1000_model_8_7_0.xlsx",
    "1000_model_18_8_24.xlsx", "1000_model_24_8_4.xlsx", "1000_model_24_9_8.xlsx", "1000_model_26_4_3.xlsx",
    "1000_model_28_9_21.xlsx", "1000_model_30_8_1.xlsx","1000_model_31_9_8.xlsx", "1000_model_35_7_20.xlsx",
    "1000_model_36_5_0.xlsx", "1000_model_38_2_0.xlsx","1000_model_38_7_24.xlsx"
]

# Chamando a função
diversity("1000_model_error", file_paths)


Arquivo 1000_model_error.xlsx salvo com sucesso!


In [10]:
import pandas as pd
import numpy as np
import itertools
import os
from tqdm import tqdm  # Para barra de progresso

def diversity(fileName, file_paths, conjunto_id):
    """
    Calcula o erro para um conjunto de redes fornecido e salva os resultados em um arquivo Excel.
    """
    dataframes = [pd.read_excel(path) for path in file_paths]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in dataframes]

    # Obter os nomes das redes
    network_names = [os.path.basename(path) for path in file_paths]

    valores = np.array(z_preds)
    Z_media = valores.mean()

    x = np.linspace(-10, 10, 32)
    y = np.linspace(-10, 10, 32)
    x, y = np.meshgrid(x, y)
    z = (x + 2 * y - 7)**2 + (2 * x + y - 5)**2
    Z = z.reshape(-1, 1)

    # Cálculo do erro
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    # Criando DataFrame para salvar no Excel
    df_error = pd.DataFrame({
    'Conjunto': [conjunto_id] * len(error.flatten()),  # Adiciona o identificador do conjunto
    'Rede': np.repeat(network_names, len(error.flatten()) // len(network_names))
    'Error': error.flatten()
     })

    # Criando pasta para salvar os arquivos (se não existir)
    output_folder = "Conjuntos_de_Redes_Erros"
    os.makedirs(output_folder, exist_ok=True)

    # Salvando a planilha com erro no diretório especificado
    file_path = os.path.join(output_folder, f"{fileName}.xlsx")
    df_error.to_excel(file_path, index=False)
    print(f'✅ Arquivo {fileName}.xlsx salvo com sucesso em {output_folder}!')

# Lista dos arquivos das 100 redes (substitua pelos nomes reais dos arquivos)
file_list = ["1000_model_4_3_13.xlsx", "1000_model_8_4_3.xlsx", "1000_model_24_8_4.xlsx",
    "1000_model_28_9_21.xlsx", "1000_model_31_9_8.xlsx", "1000_model_36_5_0.xlsx",
    "1000_model_38_2_0.xlsx", "1000_model_31_9_8.xlsx", "1000_model_36_5_0.xlsx",
    "1000_model_38_2_0.xlsx"]

# Gerando todas as combinações de 10 redes entre as 100 disponíveis
combinations_10 = list(itertools.combinations(file_list, 10))

print(f"Total de combinações: {len(combinations_10)}")

# Processando todas as combinações
for idx, file_paths in enumerate(tqdm(combinations_10, desc="Processando combinações")):
    file_name = f"error_set_{idx+1}"  # Nome do arquivo de saída
    diversity(file_name, file_paths, conjunto_id=idx+1)  # Passando o identificador do conjunto


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1440709280.py, line 34)